Claude Codeに雑にベースライン作ってとお願いしてみたら、ほぼ初手で0.8448の精度が出ました（今年のvibe codingの進歩は凄かったですね）  

改善出来るところはいくらでもあると思うので、ノートブックを共有します！


 ## 1. 設定

In [1]:
"""ベースラインモデルの設定"""

from dataclasses import dataclass
from pathlib import Path


@dataclass
class Config:
    # パス設定
    data_dir: Path = Path("../data/raw")
    image_dir: Path = Path("../data/raw/images")
    crop_dir: Path = Path("../data/processed/crops")  # 事前クロップ済み画像
    output_dir: Path = Path("../output")
    use_crops: bool = True  # 事前クロップ済み画像を使用（高速化）

    # モデル設定
    model_name: str = "resnet18"
    num_classes: int = 11  # label_id 0-10
    pretrained: bool = True
    img_size: int = 224

    # 学習設定
    batch_size: int = 128
    num_workers: int = 8
    epochs: int = 20
    lr: float = 1e-3
    weight_decay: float = 1e-4

    # 検証設定
    val_ratio: float = 0.2
    seed: int = 42

    # デバイス
    device: str = "cuda"

    def __post_init__(self):
        self.output_dir.mkdir(parents=True, exist_ok=True)

## 2. モデル定義

In [2]:
"""選手再識別用モデル定義"""

import timm
import torch
import torch.nn as nn


class PlayerClassifier(nn.Module):
    """選手再識別用CNN分類器"""

    def __init__(
        self,
        model_name: str = "resnet18",
        num_classes: int = 11,
        pretrained: bool = True,
    ):
        super().__init__()
        self.backbone = timm.create_model(
            model_name,
            pretrained=pretrained,
            num_classes=num_classes,
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.backbone(x)


def create_model(
    model_name: str = "resnet18",
    num_classes: int = 11,
    pretrained: bool = True,
) -> PlayerClassifier:
    """モデルインスタンスを作成"""
    return PlayerClassifier(
        model_name=model_name,
        num_classes=num_classes,
        pretrained=pretrained,
    )

## 3. データセットとデータローダー

In [3]:
"""選手再識別用データセット"""

import cv2
import numpy as np
import pandas as pd
import torch
from pathlib import Path
from torch.utils.data import Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2


def get_image_path(row: pd.Series, image_dir: Path) -> Path:
    """行データから画像パスを生成"""
    fname = f"{row['quarter']}__{row['angle']}__{row['session']:02d}__{row['frame']:02d}.jpg"
    return image_dir / fname


def get_train_transform(img_size: int) -> A.Compose:
    """学習用augmentationを取得"""
    return A.Compose(
        [
            A.Resize(img_size, img_size),
            A.HorizontalFlip(p=0.5),
            A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, p=0.5),
            A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1, p=0.5),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2(),
        ]
    )


def get_val_transform(img_size: int) -> A.Compose:
    """検証/テスト用変換を取得"""
    return A.Compose(
        [
            A.Resize(img_size, img_size),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2(),
        ]
    )


class PlayerDataset(Dataset):
    """選手バウンディングボックス分類用データセット"""

    def __init__(
        self,
        df: pd.DataFrame,
        image_dir: Path,
        transform: A.Compose,
        is_test: bool = False,
        cache_images: bool = False,
        crop_dir: Path = None,
    ):
        # クロップファイル検索用に元のインデックスを保持
        self.original_indices = df.index.tolist()
        self.df = df.reset_index(drop=True)
        self.image_dir = Path(image_dir)
        self.transform = transform
        self.is_test = is_test
        self.cache_images = cache_images
        self.image_cache = {}
        self.crop_dir = Path(crop_dir) if crop_dir else None
        self.use_crops = crop_dir is not None

    def __len__(self) -> int:
        return len(self.df)

    def _load_image(self, img_path: Path) -> np.ndarray:
        """キャッシュ付き画像読み込み"""
        if self.cache_images and str(img_path) in self.image_cache:
            return self.image_cache[str(img_path)]

        img = cv2.imread(str(img_path))
        if img is None:
            raise FileNotFoundError(f"画像を読み込めませんでした: {img_path}")
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        if self.cache_images:
            self.image_cache[str(img_path)] = img

        return img

    def __getitem__(self, idx: int) -> dict:
        row = self.df.iloc[idx]

        if self.use_crops:
            # 事前クロップ済み画像を直接読み込み（元のインデックスを使用）
            original_idx = self.original_indices[idx]
            crop_path = self.crop_dir / f"{original_idx}.jpg"
            crop = self._load_image(crop_path)
        else:
            # 全体画像を読み込んでクロップ
            img_path = get_image_path(row, self.image_dir)
            img = self._load_image(img_path)

            # パディング付きでバウンディングボックスをクロップ
            x, y, w, h = int(row["x"]), int(row["y"]), int(row["w"]), int(row["h"])
            img_h, img_w = img.shape[:2]

            # パディング追加（bboxサイズの10%）
            pad_w = int(w * 0.1)
            pad_h = int(h * 0.1)

            x1 = max(0, x - pad_w)
            y1 = max(0, y - pad_h)
            x2 = min(img_w, x + w + pad_w)
            y2 = min(img_h, y + h + pad_h)

            crop = img[y1:y2, x1:x2]

        # 変換を適用
        transformed = self.transform(image=crop)
        image = transformed["image"]

        result = {
            "image": image,
            "angle": row["angle"],
        }

        if not self.is_test:
            result["label"] = torch.tensor(row["label_id"], dtype=torch.long)

        return result


def create_dataloader(
    df: pd.DataFrame,
    image_dir: Path,
    img_size: int,
    batch_size: int,
    num_workers: int,
    is_train: bool = True,
    is_test: bool = False,
    crop_dir: Path = None,
) -> torch.utils.data.DataLoader:
    """データローダーを作成"""
    transform = get_train_transform(img_size) if is_train else get_val_transform(img_size)

    dataset = PlayerDataset(
        df=df,
        image_dir=image_dir,
        transform=transform,
        is_test=is_test,
        crop_dir=crop_dir,
    )

    return torch.utils.data.DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=is_train,
        num_workers=num_workers,
        pin_memory=True,
        drop_last=is_train,
        persistent_workers=num_workers > 0,
        prefetch_factor=2 if num_workers > 0 else None,
    )

## 4. 画像の事前クロップ

In [4]:
"""学習高速化のための画像事前クロップ処理"""

import cv2
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor, as_completed
import multiprocessing


def get_image_path(row: pd.Series, image_dir: Path) -> Path:
    """行データから画像パスを生成"""
    fname = f"{row['quarter']}__{row['angle']}__{row['session']:02d}__{row['frame']:02d}.jpg"
    return image_dir / fname


def process_single_crop(args: tuple) -> tuple[int, bool]:
    """単一クロップを処理。(idx, success)を返す"""
    idx, row, image_dir, output_dir, padding_ratio = args

    try:
        img_path = get_image_path(row, image_dir)
        img = cv2.imread(str(img_path))

        if img is None:
            return idx, False

        # パディング付きでbboxを取得
        x, y, w, h = int(row["x"]), int(row["y"]), int(row["w"]), int(row["h"])
        img_h, img_w = img.shape[:2]

        pad_w = int(w * padding_ratio)
        pad_h = int(h * padding_ratio)

        x1 = max(0, x - pad_w)
        y1 = max(0, y - pad_h)
        x2 = min(img_w, x + w + pad_w)
        y2 = min(img_h, y + h + pad_h)

        crop = img[y1:y2, x1:x2]

        # クロップを保存
        output_path = output_dir / f"{idx}.jpg"
        cv2.imwrite(str(output_path), crop, [cv2.IMWRITE_JPEG_QUALITY, 95])

        return idx, True
    except Exception as e:
        print(f"idx {idx} の処理中にエラー: {e}")
        return idx, False


def preprocess_crops(
    csv_path: Path,
    image_dir: Path,
    output_dir: Path,
    padding_ratio: float = 0.1,
    num_workers: int = None,
):
    """全バウンディングボックスを事前クロップして保存"""
    if num_workers is None:
        num_workers = multiprocessing.cpu_count()

    output_dir.mkdir(parents=True, exist_ok=True)

    df = pd.read_csv(csv_path)
    print(f"{len(df)} サンプルを {num_workers} ワーカーで処理中...")

    # 並列処理用の引数を準備
    args_list = [(idx, row, image_dir, output_dir, padding_ratio) for idx, row in df.iterrows()]

    # 並列処理
    success_count = 0
    failed_indices = []

    with ProcessPoolExecutor(max_workers=num_workers) as executor:
        futures = {executor.submit(process_single_crop, args): args[0] for args in args_list}

        for future in tqdm(as_completed(futures), total=len(futures), desc="クロップ中"):
            idx, success = future.result()
            if success:
                success_count += 1
            else:
                failed_indices.append(idx)

    print(f"完了: {success_count}/{len(df)} クロップを保存")
    if failed_indices:
        print(f"失敗したインデックス: {failed_indices[:10]}...")

In [5]:
# 前処理を実行（実行する場合はコメントを解除）
cfg = Config()

# 学習データを処理
preprocess_crops(
    csv_path=cfg.data_dir / "train_meta.csv",
    image_dir=cfg.image_dir,
    output_dir=cfg.crop_dir / "train",
)

# テストデータが存在する場合は処理
test_csv = cfg.data_dir / "test_meta.csv"
if test_csv.exists():
    preprocess_crops(
        csv_path=test_csv,
        image_dir=cfg.image_dir,
        output_dir=cfg.crop_dir / "test",
    )

24920 サンプルを 32 ワーカーで処理中...


クロップ中: 100%|██████████| 24920/24920 [01:12<00:00, 344.71it/s]


完了: 24920/24920 クロップを保存
7500 サンプルを 32 ワーカーで処理中...


クロップ中: 100%|██████████| 7500/7500 [00:21<00:00, 354.82it/s]

完了: 7500/7500 クロップを保存


## 5. 学習

In [6]:
"""選手再識別の学習スクリプト"""

from sklearn.model_selection import StratifiedGroupKFold


def train_one_epoch(
    model: nn.Module,
    loader: torch.utils.data.DataLoader,
    criterion: nn.Module,
    optimizer: torch.optim.Optimizer,
    device: str,
) -> float:
    """1エポック分の学習"""
    model.train()
    total_loss = 0.0
    num_samples = 0

    pbar = tqdm(loader, desc="学習")
    for batch in pbar:
        images = batch["image"].to(device)
        labels = batch["label"].to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * images.size(0)
        num_samples += images.size(0)
        pbar.set_postfix({"loss": total_loss / num_samples})

    return total_loss / num_samples


@torch.no_grad()
def validate(
    model: nn.Module,
    loader: torch.utils.data.DataLoader,
    criterion: nn.Module,
    device: str,
) -> tuple[float, float]:
    """モデルの検証"""
    model.eval()
    total_loss = 0.0
    correct = 0
    num_samples = 0

    for batch in tqdm(loader, desc="検証"):
        images = batch["image"].to(device)
        labels = batch["label"].to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        total_loss += loss.item() * images.size(0)
        _, preds = outputs.max(1)
        correct += (preds == labels).sum().item()
        num_samples += images.size(0)

    return total_loss / num_samples, correct / num_samples


def train_main():
    cfg = Config()
    print(f"設定: {cfg}")

    # シード設定
    torch.manual_seed(cfg.seed)

    # データ読み込み
    train_df = pd.read_csv(cfg.data_dir / "train_meta.csv")
    print(f"学習サンプル数: {len(train_df)}")

    # quarterでグループ分割（データリーケージ防止のためゲーム単位で分割）
    train_df["group"] = train_df["quarter"]
    sgkf = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=cfg.seed)

    # 最初のfoldを検証用に使用
    train_idx, val_idx = next(sgkf.split(train_df, train_df["label_id"], train_df["group"]))
    train_data = train_df.iloc[train_idx]
    val_data = train_df.iloc[val_idx]

    print(f"学習: {len(train_data)}, 検証: {len(val_data)}")
    print(f"検証quarters: {val_data['quarter'].unique()}")

    # データローダー作成
    crop_dir = cfg.crop_dir / "train" if cfg.use_crops else None
    train_loader = create_dataloader(
        train_data, cfg.image_dir, cfg.img_size, cfg.batch_size, cfg.num_workers, is_train=True, crop_dir=crop_dir
    )
    val_loader = create_dataloader(
        val_data, cfg.image_dir, cfg.img_size, cfg.batch_size, cfg.num_workers, is_train=False, crop_dir=crop_dir
    )

    # モデル作成
    model = create_model(cfg.model_name, cfg.num_classes, cfg.pretrained)
    model = model.to(cfg.device)
    print(f"モデル: {cfg.model_name}")

    # 損失関数とオプティマイザ
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=cfg.lr, weight_decay=cfg.weight_decay)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=cfg.epochs)

    # 学習ループ
    best_acc = 0.0
    for epoch in range(cfg.epochs):
        print(f"\nエポック {epoch + 1}/{cfg.epochs}")

        train_loss = train_one_epoch(model, train_loader, criterion, optimizer, cfg.device)
        val_loss, val_acc = validate(model, val_loader, criterion, cfg.device)
        scheduler.step()

        print(f"学習Loss: {train_loss:.4f}, 検証Loss: {val_loss:.4f}, 検証Acc: {val_acc:.4f}")

        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), cfg.output_dir / "best_model.pth")
            print(f"ベストモデルを保存! Acc: {best_acc:.4f}")

    print(f"\n学習完了。ベスト精度: {best_acc:.4f}")

In [7]:
train_main()

設定: Config(data_dir=PosixPath('/kaggle/input/atmacup22'), image_dir=PosixPath('/kaggle/input/atmacup22/images'), crop_dir=PosixPath('/kaggle/dataset/crops'), output_dir=PosixPath('/kaggle/output'), use_crops=True, model_name='resnet18', num_classes=11, pretrained=True, img_size=224, batch_size=128, num_workers=8, epochs=20, lr=0.001, weight_decay=0.0001, val_ratio=0.2, seed=42, device='cuda')
学習サンプル数: 24920
学習: 19660, 検証: 5260
検証quarters: ['Q1-000' 'Q1-003' 'Q2-000' 'Q2-004' 'Q2-012' 'Q2-016']


/kaggle/.venv/lib/python3.12/site-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


モデル: resnet18

エポック 1/20


検証: 100%|██████████| 42/42 [00:02<00:00, 14.57it/s]


学習Loss: 0.7054, 検証Loss: 0.2291, 検証Acc: 0.9287
ベストモデルを保存! Acc: 0.9287

エポック 2/20


...

学習Loss: 0.0038, 検証Loss: 0.0733, 検証Acc: 0.9819
ベストモデルを保存! Acc: 0.9819

エポック 19/20


検証: 100%|██████████| 42/42 [00:02<00:00, 14.81it/s]


学習Loss: 0.0044, 検証Loss: 0.0741, 検証Acc: 0.9816

エポック 20/20


検証: 100%|██████████| 42/42 [00:02<00:00, 14.84it/s]


学習Loss: 0.0034, 検証Loss: 0.0751, 検証Acc: 0.9823
ベストモデルを保存! Acc: 0.9823

学習完了。ベスト精度: 0.9823


## 6. 推論

In [8]:
"""submission用推論スクリプト"""


@torch.no_grad()
def predict(
    model: torch.nn.Module,
    loader: torch.utils.data.DataLoader,
    device: str,
) -> list[int]:
    """予測を生成"""
    model.eval()
    predictions = []

    for batch in tqdm(loader, desc="予測"):
        images = batch["image"].to(device)
        outputs = model(images)
        _, preds = outputs.max(1)
        predictions.extend(preds.cpu().numpy().tolist())

    return predictions


def inference_main():
    cfg = Config()

    # テストデータ読み込み
    test_side = pd.read_csv(cfg.data_dir / "test_meta.csv")
    test_top = pd.read_csv(cfg.data_dir / "test_top_meta.csv")

    print(f"テストsideサンプル数: {len(test_side)}")
    print(f"テストtopサンプル数: {len(test_top)}")

    # モデル作成と重み読み込み
    model = create_model(cfg.model_name, cfg.num_classes, pretrained=False)
    model_path = cfg.output_dir / "best_model.pth"

    if not model_path.exists():
        print(f"モデルが見つかりません: {model_path}")
        print("先にtrain.pyを実行してください!")
        return

    model.load_state_dict(torch.load(model_path, map_location=cfg.device, weights_only=True))
    model = model.to(cfg.device)
    print(f"モデルを読み込みました: {model_path}")

    # データローダー作成
    test_side_loader = create_dataloader(
        test_side, cfg.image_dir, cfg.img_size, cfg.batch_size, cfg.num_workers, is_train=False, is_test=True
    )
    test_top_loader = create_dataloader(
        test_top, cfg.image_dir, cfg.img_size, cfg.batch_size, cfg.num_workers, is_train=False, is_test=True
    )

    # 予測
    preds_side = predict(model, test_side_loader, cfg.device)
    preds_top = predict(model, test_top_loader, cfg.device)

    print(f"side予測数: {len(preds_side)}")
    print(f"top予測数: {len(preds_top)}")

    # submission作成（sample_submissionに従いsideのみ）
    submission = pd.DataFrame({"label_id": preds_side})
    submission_path = cfg.output_dir / "submission.csv"
    submission.to_csv(submission_path, index=False)
    print(f"submissionを保存: {submission_path}")

    # 全予測も保存
    full_submission = pd.DataFrame({"label_id": preds_side + preds_top})
    full_submission.to_csv(cfg.output_dir / "submission_full.csv", index=False)

    # 予測分布を表示
    print("\n予測分布 (side):")
    print(pd.Series(preds_side).value_counts().sort_index())

In [9]:
inference_main()

テストsideサンプル数: 7500
テストtopサンプル数: 1480
モデルを読み込みました: /kaggle/output/best_model.pth


予測: 100%|██████████| 12/12 [00:10<00:00,  1.19it/s]


side予測数: 7500
top予測数: 1480
submissionを保存: /kaggle/output/submission.csv

予測分布 (side):
0       38
1      754
2      759
3      770
4      766
5      741
6      735
7      752
8       48
9     1277
10     860
Name: count, dtype: int64
